# Make an EPN-TAP data table for PLANMAP packages

We want to go inside Planmap packages and extract information for each data product in `vector` and `raster` directories.
This information will be used, initially, to populate VESPA EPN-TAP tables.
In particular, we're interested in filling the following columns<sup>*</sup>:

* `access_url`: data product URL
* `c1max`: max longitude (positive east, 0-360)
* `c1min`: min longitude (positive east, 0-360)
* `c2max`: max latitude
* `c2min`: min latitude
* `s_region`: map footprint
* `creation_date`: package creation date
* `dataproduct_type`: options are `im`
* `granule_gid`: package-id (PM_ID)
* `granule_uid`: data product name
* `modification_date`: package update date (same as creation data, if in doubt)
* `obs_id`: data product name
* `release_date`: package release date (same as creation date, if in doubt)
* `service_title`: `planmap`
* `spatial_frame_type`: `body`
* `target_class`: `planet`
* `target_name`: options are `mars`, `mercury`, `moon`
* `thumbnail_url`: thumbnail (jpeg) url

<sup>*</sup> About EPN-TAP columns: https://voparis-wiki.obspm.fr/display/VES/EPN-TAP+v2+parameter+description

### Software/libraries we're using here:

* Python 3.7.8
* rasterio 1.2.0
* shapely 1.7.1
* fiona 1.8.18

In [1]:
from dataclasses import make_dataclass, field, asdict

epntap_fields = [
    ('granule_uid', str),
    ('granule_gid', str),
    ('obs_id', str),
    ('dataproduct_type', str, field(default='')),
    ('access_url', str, field(default='')),
    ('c1max', float, field(default=360)),
    ('c1min', float, field(default=0)),
    ('c2max', float, field(default=90)),
    ('c2min', float, field(default=-90)),
    ('s_region', str, field(default='')),
    ('creation_date', str, field(default='2019-12-31')),
    ('modification_date', str, field(default='2019-12-31')),
    ('release_date', str, field(default='2019-12-31')),
    ('service_title', str, field(default='planmap')),
    ('spatial_frame_type', str, field(default='body')),
    ('target_class', str, field(default='planet')),
    ('target_name', str, field(default='')),
    ('thumbnail_url', str, field(default=''))
]

EPNTAPRecord = make_dataclass('EPNTAPRecord', epntap_fields)

EPNTAPRecordRaster = make_dataclass('EPNTAPRecordRaster', 
                                    [('dataproduct_type', str, 'im')],
                                    bases=(EPNTAPRecord,))

EPNTAPRecordVector = make_dataclass('EPNTAPRecordVector', 
                                    [('dataproduct_type', str, 'sv')],
                                    bases=(EPNTAPRecord,))

In [2]:
import pathlib

import fiona
import shapely
import rasterio


crs = {
    'moon': '+proj=longlat +a=1737400 +b=1737400 +no_defs',
    'mars': '+proj=longlat +a=3396190 +b=3376200 +no_defs'
}

def bbox_shift_0360(bbox):
    c1min = bbox['c1min'] + 360 if bbox['c1min'] < 0 else bbox['c1min']
    c1max = bbox['c1max'] + 360 if bbox['c1max'] < 0 else bbox['c1max']
    bbox['c1min'] = c1min
    bbox['c1max'] = c1max
    return bbox
    
def bounding_box_raster(tif, body):
    from rasterio import warp
    img = rasterio.open(tif)
    dst_crs = crs[body]
    left, bottom, right, top = warp.transform_bounds(img.crs, dst_crs, *img.bounds)
    bbox = {
        'c1min': left,
        'c1max': right,
        'c2min': bottom,
        'c2max': top
    }
    return bbox_shift_0360(bbox)
    
def bounding_box_vector(gpkg, body):
    from rasterio import warp
    f = fiona.open('PM-MAR-MS-Arsinoes/vector/PM-MAR-MS-Arsinoes.gpkg')
    dst_crs = crs[body]
    left, bottom, right, top = warp.transform_bounds(f.crs, dst_crs, *f.bounds)
    bbox = {
        'c1min': left,
        'c1max': right,
        'c2min': bottom,
        'c2max': top
    }
    return bbox_shift_0360(bbox)

def s_region_bbox(bbox):
    from shapely import geometry
    coords = [
        (bbox['c1min'],bbox['c2min']),
        (bbox['c1min'],bbox['c2max']),
        (bbox['c1max'],bbox['c2max']),
        (bbox['c1max'],bbox['c2min']),
    ]
    pol = geometry.Polygon(coords)
    return pol

def init_path(path):
    p = pathlib.Path(path)
    if not (p.exists() and p.is_dir()):
        return None
    return p
    
    
class PmPkg:
    _rasters = []
    _vectors = []
    
    def __init__(self, path='./PM-MAR-C-Arsinoes/'):
        _path = init_path(path)
        assert _path is not None, "Path '{!s}' does not seem to exist".format(path)
        self._path = _path
        self._pmid = _path.name
        
    @property
    def pmid(self):
        return self._pmid
    
    def parse_readme(self, param):
        with open(self._path / 'README.md', 'r') as fp:
            _lines = fp.readlines()
        for _l in _lines:
            _l = _l.strip().lower()
            if param.lower() in _l:
                return _l.split('|')[2].strip()
        return ''
        
    def proc_raster_dir(self):
        rasters = []
        path = self._path / 'raster'
        if path.exists and path.is_dir():
            for _f in path.iterdir():
                if _f.suffix == '.tif' or _f.suffix == '.tiff':
                    filename = path / _f.name
                    granule_uid = _f.stem
                    granule_gid = self.pmid
                    obs_id = granule_uid.replace(granule_gid + '_', '')
                    target_name = self.parse_readme('target body')
                    assert target_name in ('mars', 'mercury', 'moon'), "Expecting 'target_name' in 'mars', 'mercury', or 'moon'"
                    access_url = '/'.join(['https://data.planmap.eu/pub', target_name, self.pmid, 'raster', _f.name])
                    thumbnail_url = '.'.join(access_url.split('.')[:-1] + ['jpg'])
                    bbox = bounding_box_raster(filename, target_name)
                    s_region = s_region_bbox(bbox)
                    _raster = EPNTAPRecordRaster(granule_uid = granule_uid,
                                                 obs_id = obs_id,
                                                 granule_gid = granule_gid,
                                                 access_url = access_url,
                                                 target_name = target_name,
                                                 thumbnail_url = thumbnail_url,
                                                 c1min = bbox['c1min'],
                                                 c1max = bbox['c1max'],
                                                 c2min = bbox['c2min'],
                                                 c2max = bbox['c2max'],
                                                 s_region = s_region
                                                )
                    rasters.append(_raster)
            self._rasters = rasters
        
    def proc_vector_dir(self):
        vectors = []
        path = self._path / 'vector'
        if path.exists and path.is_dir():
            for _f in path.iterdir():
                if _f.suffix == '.gpkg':
                    filename = path / _f.name
                    granule_uid = _f.stem
                    granule_gid = self.pmid
                    obs_id = granule_uid
                    target_name = self.parse_readme('target body')
                    assert target_name in ('mars', 'mercury', 'moon'), "Expecting 'target_name' in 'mars', 'mercury', or 'moon'"
                    access_url = '/'.join(['https://data.planmap.eu/pub', target_name, self.pmid, 'vector', _f.name])
                    bbox = bounding_box_vector(filename, target_name)
                    _vector = EPNTAPRecordVector(granule_uid = granule_uid,
                                                 obs_id = obs_id,
                                                 granule_gid = granule_gid,
                                                 access_url = access_url,
                                                 target_name = target_name,
                                                 c1min = bbox['c1min'],
                                                 c1max = bbox['c1max'],
                                                 c2min = bbox['c2min'],
                                                 c2max = bbox['c2max']
                                                )
                    vectors.append(_vector)
            self._vectors = vectors                
    
    def proc_data(self):
        self.proc_raster_dir()
        self.proc_vector_dir()

    def to_epntap(self):
        epn_records = []
        for epn_rec in self._rasters:
            epn_records.append(asdict(epn_rec))
        for epn_rec in self._vectors:
            epn_records.append(asdict(epn_rec))
        return epn_records
    

In [4]:
pkg = PmPkg('../PM-MAR-C-Arsinoes/')

pkg.proc_raster_dir()

epntap_records = pkg.to_epntap()

AssertionError: Path '../PM-MAR-C-Arsinoes/' does not seem to exist

In [ ]:
import pandas

pandas.DataFrame(epntap_records)

In [ ]:
pkg = PmPkg('./PM-MAR-MS-Arsinoes/')

pkg.proc_data()

epntap_records = pkg.to_epntap()

pandas.DataFrame(epntap_records)

In [ ]:
from IPython.display import display

import pathlib
cwd = pathlib.Path('.')

df = None
for _dir in cwd.iterdir():
    if not (_dir.is_dir() and _dir.name[:2] == 'PM'):
        continue
    print(_dir)
    try:
        pkg = PmPkg(_dir)
        pkg.proc_data()
        epntap_records = pkg.to_epntap()
    except Exception as err:
        print(err)
    _df = pandas.DataFrame(epntap_records)
    display(_df)
    if df is None:
        df = _df
    else:
        df = pandas.concat([df,_df]).reset_index(drop=True)

In [ ]:
df

In [ ]:
df.to_csv('/Users/chbrandt/Coisas/repos/VESPA/services/planmap/data/data.csv')

In [5]:
%ls

make_epntap_data_table.ipynb


In [6]:
%ls ..

data/ docs/ q.rd*
